<style>
@font-face {font-family: "B Nazanin"; src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot"); src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff") format("woff"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.svg#B Nazanin") format("svg"); }
    </style>
<div dir="rtl" align="center" style ='font-family: "B Nazanin";'>
    <h1>
        تمرین سوم درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    ما در این تمرین ترک "تکمیل مصراع دوم با رعایت وزن شعر" را انتخاب کردیم. در این تمرین قصد داریم به کمک مدل‌های زبانی مناسب، چند کلمه به عنوان یک مصرع از بیت را به عنوان ورودی بگیریم و چند کلمه به عنوان مصرع دوم این بیت را خروجی دهیم. در این تمرین ما از وبسایت گنجور برای جمع‌آوری داده کمک گرفتیم و از چندین قالب شعری استفاده کردیم، زیرا قافیه داشتن یا نداشتن به عنوان ورودی داده می‌شود. اگر نیاز به هم‌قافیه بودن دو مصراع بود، از قالب مثنوی نظیر شاهنماه‌ی فردوسی کمک می‌گیریم؛ و اگر نیاز به هم‌قافیه بودن دو مصراع نبود می‌توانیم از قالب قطعه استفاده کنیم. <br>
    برای ساخت چنین سیستمی از چند مدل زبانی استفاده می‌کنیم: مدل n-gram، مدل encoder-decoder با یک شبکه LSTM به عنوان encoder، و مدلی پیرو مکانیسم توجه.
</div>

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 1. دوباره‌نویسی داده </h3>
    از آنجایی که در داده‌ی گرفته شده، ابتدا و انتهای ابیات و مصراع‌ها مشخص نبود، به کمک کد زیر این داده را بازنویسی کردیم. ابتدا و انتهای ابیات با __BOB__ و __EOB__ و ابتدا و انتهای مصراع‌ها با __BOM__ و __EOM__ مشخص گردید.
</div>

In [1]:
path = 'ferdousi_norm.txt'

beyt_file = ''
with open(path, 'r', encoding="utf-8") as fp:
    lines = fp.readlines()
    for i in range(0, len(lines) - 1, 2):
        mesra1 = '__BOM__ ' + lines[i].strip() + ' __EOM__ '
        mesra2 = '__BOM__ ' + lines[i + 1].strip() + ' __EOM__'
        b = '__BOB__ ' + mesra1.strip() + ' ' + mesra2.strip() + ' __EOB__\n'
        beyt_file += b
        
with open('f_beyt.txt', 'w', encoding="utf-8") as fp:
    fp.write(beyt_file)